# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [93]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [94]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,19.01,64,80,4.53,PN,2024-08-28
1,1,port-aux-francais,-49.3500,70.2167,5.53,97,100,13.67,TF,2024-08-28
2,2,imbituba,-28.2400,-48.6703,10.14,86,25,1.57,BR,2024-08-28
3,3,waitangi,-43.9535,-176.5597,11.01,96,100,4.92,NZ,2024-08-28
4,4,campbell river,50.0163,-125.2446,13.58,85,0,3.48,CA,2024-08-28


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [95]:
%%capture --no-display

# Configure the map plot
plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo= True,
    tiles = 'OSM',
    frame_width = 1000,
    frame_height = 800,
    size = 'Humidity',
    color = 'City',
)

# Display the map
plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [96]:
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,19.01,64,80,4.53,PN,2024-08-28
1,1,port-aux-francais,-49.3500,70.2167,5.53,97,100,13.67,TF,2024-08-28
2,2,imbituba,-28.2400,-48.6703,10.14,86,25,1.57,BR,2024-08-28
3,3,waitangi,-43.9535,-176.5597,11.01,96,100,4.92,NZ,2024-08-28
4,4,campbell river,50.0163,-125.2446,13.58,85,0,3.48,CA,2024-08-28
...,...,...,...,...,...,...,...,...,...,...
604,604,guarapari,-20.6667,-40.4975,15.96,82,18,2.85,BR,2024-08-28
605,605,kemin,42.7846,75.6892,29.45,22,51,3.05,KG,2024-08-28
606,606,tobelo,1.7284,128.0095,28.54,75,55,0.58,ID,2024-08-28
607,607,kingscliff,-28.2667,153.5667,23.89,78,0,7.20,AU,2024-08-28


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [97]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df[(city_data_df.Lat < 40) & (city_data_df.Lat > -20)
                             & (city_data_df.Humidity < 60) & (city_data_df.Cloudiness < 60)]

# Drop any rows with null values
city_data_df.count().notnull

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,delano,35.7688,-119.2470,23.00,56,0,1.54,US,2024-08-28
27,27,ghazni,33.5536,68.4269,23.00,32,0,2.22,AF,2024-08-28
28,28,jinchang,38.4953,102.1739,26.63,42,23,3.13,CN,2024-08-28
72,72,robore,-18.3333,-59.7500,18.78,36,1,1.52,BO,2024-08-28
83,83,avra valley,32.4379,-111.3154,32.55,36,59,4.57,US,2024-08-28
...,...,...,...,...,...,...,...,...,...,...
589,589,zarand,30.8127,56.5640,31.28,12,0,1.43,IR,2024-08-28
593,593,antsiranana,-12.2787,49.2917,25.42,53,20,8.23,MG,2024-08-28
597,597,cuatro cienegas de carranza,26.9833,-102.0833,25.81,58,0,1.93,MX,2024-08-28
599,599,pailon,-17.6500,-62.7500,17.03,46,30,2.28,BO,2024-08-28


### Step 3: Create a new DataFrame called `hotel_df`.

In [98]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()
hotel_df = hotel_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
5,delano,US,35.7688,-119.2470,56,
27,ghazni,AF,33.5536,68.4269,32,
28,jinchang,CN,38.4953,102.1739,42,
72,robore,BO,-18.3333,-59.7500,36,
83,avra valley,US,32.4379,-111.3154,36,
...,...,...,...,...,...,...
589,zarand,IR,30.8127,56.5640,12,
593,antsiranana,MG,-12.2787,49.2917,53,
597,cuatro cienegas de carranza,MX,26.9833,-102.0833,58,
599,pailon,BO,-17.6500,-62.7500,46,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [112]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,  
    'categories': "accommodation.hotel",
    "limit": 1,
    'radius': radius
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
# hotel_df

Starting hotel search
delano - nearest hotel: Holiday Inn Express Delano Hwy 99
ghazni - nearest hotel: No hotel found
jinchang - nearest hotel: No hotel found
robore - nearest hotel: Lajas
avra valley - nearest hotel: No hotel found
ad dilam - nearest hotel: No hotel found
moapa valley - nearest hotel: No hotel found
qamdo - nearest hotel: 银杏商务宾馆
kosciusko - nearest hotel: President's Inn
al jawf - nearest hotel: No hotel found
herencia - nearest hotel: No hotel found
elk city - nearest hotel: King's Inn
jincheng - nearest hotel: 德豪宾馆
lupane - nearest hotel: No hotel found
aswan - nearest hotel: Oscar Hotel
watampone - nearest hotel: Hotel Rio Rita
formosa do rio preto - nearest hotel: Pousada Universo - Formosa do Rio Preto BA
mashhad - nearest hotel: هتل تعاون
dadukou - nearest hotel: 锦江之星酒店
sur - nearest hotel: Sur Hotel
karema - nearest hotel: No hotel found
siwa oasis - nearest hotel: Flower of the city hotel
laguna - nearest hotel: Holiday Inn Express & Suites
gharavuti - neares

In [114]:
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
5,delano,US,35.7688,-119.2470,56,Holiday Inn Express Delano Hwy 99
27,ghazni,AF,33.5536,68.4269,32,No hotel found
28,jinchang,CN,38.4953,102.1739,42,No hotel found
72,robore,BO,-18.3333,-59.7500,36,Lajas
83,avra valley,US,32.4379,-111.3154,36,No hotel found
...,...,...,...,...,...,...
589,zarand,IR,30.8127,56.5640,12,No hotel found
593,antsiranana,MG,-12.2787,49.2917,53,Central
597,cuatro cienegas de carranza,MX,26.9833,-102.0833,58,No hotel found
599,pailon,BO,-17.6500,-62.7500,46,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [117]:
%%capture --no-display

# Configure the map plot
plot_2 = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo= True,
    tiles = 'OSM',
    frame_width = 800,
    frame_height = 600,
    size = 'Humidity',
    color = 'City',
    hover_cols=['Hotel Name', 'Country']
)

# Display the map
plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)